# SIMPLE NEURAL NETWORK TO CARTIGORIZE MUSIC
#### author: Rhema Ike

In [96]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [3]:
import os
import random

In [104]:
NAME = "MUSIC_CAT_NN_1"
dir_name = os.path.join("logs", NAME)
tensorboard = Tensorboard = TensorBoard(log_dir=dir_name)

In [64]:
# Load Data

DATA_DIR = "DataSets"
file_name = "SongDataSetLess_4000.xlsx"

file_name = os.path.join(DATA_DIR, file_name)

df = pd.read_excel(file_name)
df.drop(["name","artist"], axis=1,inplace=True)


PLAYLIST_LEN = 1000
target = ([1]* PLAYLIST_LEN) + ([2]* PLAYLIST_LEN) + ([3]* PLAYLIST_LEN) + ([4]* PLAYLIST_LEN)
target_df = pd.DataFrame(target, columns=["target"])

df = pd.concat([df,target_df], axis=1)

In [60]:
df.iloc[2400]

name                   Believer
artist              Crazy Fluke
key                          11
liveness                   0.22
instrumentalness          0.874
loudness                 -7.407
mode                          0
speechiness              0.0813
tempo                   122.988
valence                   0.684
danceability              0.723
energy                    0.869
acousticness           0.000404
target                        3
Name: 2400, dtype: object

In [61]:
import sklearn as sk
import numpy as np

In [62]:
from sklearn import preprocessing as preproc

In [63]:
def preproc_df(df):
    for col in df.columns:
        if col in ["key", "loudness", "tempo"]:
            df[col] = preproc.scale(df[col].values)

    df.dropna(inplace=True)
    data = []
    for row in df.values:
        data.append([np.array(row[:-1]),row[-1]])
        
    random.shuffle(data)
        
    X_data = []
    y_data = []
    
    for X,y in data:
        X_data.append(X)
        y_data.append(y-1)
    
        
    return np.array(X_data), np.array(y_data)
    

In [65]:
X,y = preproc_df(df)

In [101]:
# Making Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(60, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(60, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [105]:
model.fit(X,y,epochs=10,validation_split=0.2,  callbacks = [tensorboard])

Train on 3200 samples, validate on 800 samples
Epoch 1/10
3200/3200 [==============================] - 1s 172us/sample - loss: 0.8435 - accuracy: 0.6803 - val_loss: 0.5521 - val_accuracy: 0.7775
Epoch 2/10
3200/3200 [==============================] - 0s 50us/sample - loss: 0.4970 - accuracy: 0.8019 - val_loss: 0.4662 - val_accuracy: 0.8075
Epoch 3/10
3200/3200 [==============================] - 0s 60us/sample - loss: 0.4414 - accuracy: 0.8256 - val_loss: 0.4265 - val_accuracy: 0.8300
Epoch 4/10
3200/3200 [==============================] - 0s 58us/sample - loss: 0.4081 - accuracy: 0.8416 - val_loss: 0.4137 - val_accuracy: 0.8388
Epoch 5/10
3200/3200 [==============================] - 0s 60us/sample - loss: 0.3886 - accuracy: 0.8456 - val_loss: 0.4041 - val_accuracy: 0.8462
Epoch 6/10
3200/3200 [==============================] - 0s 50us/sample - loss: 0.3775 - accuracy: 0.8522 - val_loss: 0.4404 - val_accuracy: 0.8263
Epoch 7/10
3200/3200 [==============================] - 0s 52us/sample

In [68]:
# Save Model

MODEL_DIR = "models"
file_name = "simp_neural_net_2l_60n"

file_name = os.path.join(MODEL_DIR, file_name)
model.save(file_name)

INFO:tensorflow:Assets written to: models\simp_neural_net_2l_60n\assets


In [83]:
# Testing Model
Song_categorizer_model = tf.keras.models.load_model(file_name)
predictions = Song_categorizer_model.predict(X)
# np.array(X[0])

In [93]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_layer_names=True,
    rankdir='TB', expand_nested=False, dpi=96
)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
